# Open Neural Network Exchange
# **ONNX**

In [ ]:
import numpy as np
import os, time
from transformers import TextClassificationPipeline, AutoTokenizer, AutoModelForSequenceClassification 
import torch, onnx
import onnxruntime
from onnxruntime import InferenceSession, SessionOptions
from onnxruntime.capi.onnxruntime_pybind11_state import InvalidArgument

In [2]:
model_name = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = False)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 3)
pipeline =  TextClassificationPipeline(model=model, tokenizer=tokenizer, max_length=64, truncation=True, padding='max_length', top_k=None)

In [3]:
t_start = time.time()
news_list = ["How the GBTC premium commerce ruined Barry Silbert, his DCG empire and took crypto lending platforms with them",
             "MicroStrategy’s Bitcoin Strategy Sets Tongues Wagging Even As It Doubles Down On BTC Purchases ⋆ ZyCrypto",
             "How the GBTC premium trade ruined Barry Silbert, his DCG empire and took crypto lending platforms with them",
             "Bitcoin Holders To Expect More Difficulties As Data Point To Looming BTC Price Drop",
             "Bitcoin Breaks Past $17,000 Barrier – Will BTC Also Breach 4% Weekly Run?",
             "Bitcoin Price Today 9 Jan: BTC Increases By 1.79% Painting The Chart Green",
             "Bitcoin: This is what large investor and retail interest can do for BTC over time"                          
]
for n in news_list:
    print(pipeline(n),"\n")
    # input_ids = torch.tensor(tokenizer.encode(n)).unsqueeze(0)
    # outputs = model(input_ids)
    # print(outputs)
    
print(time.time() - t_start)   


[[{'label': 'Neutral', 'score': 0.6913473010063171}, {'label': 'Bullish', 'score': 0.308287113904953}, {'label': 'Bearish', 'score': 0.00036554550752043724}]] 

[[{'label': 'Neutral', 'score': 0.5331341624259949}, {'label': 'Bullish', 'score': 0.4627279043197632}, {'label': 'Bearish', 'score': 0.004137895535677671}]] 

[[{'label': 'Neutral', 'score': 0.8244197964668274}, {'label': 'Bullish', 'score': 0.17524367570877075}, {'label': 'Bearish', 'score': 0.00033660337794572115}]] 

[[{'label': 'Bullish', 'score': 0.5207508206367493}, {'label': 'Neutral', 'score': 0.4782118797302246}, {'label': 'Bearish', 'score': 0.001037305104546249}]] 

[[{'label': 'Bullish', 'score': 0.54047691822052}, {'label': 'Neutral', 'score': 0.44001147150993347}, {'label': 'Bearish', 'score': 0.019511574879288673}]] 

[[{'label': 'Neutral', 'score': 0.5612754225730896}, {'label': 'Bullish', 'score': 0.43397819995880127}, {'label': 'Bearish', 'score': 0.004746420308947563}]] 

[[{'label': 'Neutral', 'score': 0.38

In [4]:
dummy_model_input = tokenizer(
        "текст для конвертации",
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    ).to("cpu")
torch.onnx.export(
    model,
    dummy_model_input["input_ids"],
    "bert.onnx",
    opset_version=12,
    input_names=["input_ids"],
    output_names=["output"],
    dynamic_axes={
        "input_ids": {
            0: "batch_size",
            1: "sequence_len"
        },
        "output": {
            0: "batch_size"
        }
    }
)

In [5]:

torch.set_num_threads(1)

options = SessionOptions()
options.graph_optimization_level = onnxruntime.GraphOptimizationLevel.ORT_ENABLE_ALL
options.intra_op_num_threads = 1
session = InferenceSession("bert.onnx", options, providers=["CPUExecutionProvider"])
session.disable_fallback()



In [6]:
def softmax(logits):
    logits = np.exp(logits)
    s = sum(logits)
    for i in range(3):
        logits[i]=logits[i]/s
    return logits

In [7]:
t_start = time.time()
for n in news_list:        
    # inputs = tokenizer(
    #     n,
    #     padding="max_length",
    #     truncation=True,
    #     max_length=200,
    #     return_tensors="pt"
    # ).to('cpu')
    inputs = tokenizer(n)
    
    #input_feed = {"input_ids":inputs["input_ids"].detach().numpy()}
    input_feed = {"input_ids":np.array(inputs["input_ids"]).reshape(1,-1)}
    try:
        outputs = session.run(
             output_names=["output"],
             input_feed=input_feed
        )[0][0]
    except (RuntimeError, InvalidArgument) as e:
        print("Error with shape {}  - {}".format(input_feed["input_ids"].shape, e))
    
    sm = softmax(outputs)
    print("{:.6f}, {:.6f}, {:.6f}".format(sm[0], sm[1], sm[2]))
    
print(time.time() - t_start)
#output = session.run(output_names = "run_hidden_last", input_feed=dict(inputs))

0.000366, 0.691347, 0.308287
0.004138, 0.533135, 0.462727
0.000337, 0.824420, 0.175244
0.001037, 0.478212, 0.520750
0.019511, 0.440012, 0.540477
0.004746, 0.561276, 0.433977
0.343346, 0.387683, 0.268971
0.547950029373169


In [20]:
# dummy_model_input = tokenizer(
#     "текст для конвертации",
#     padding="max_length",
#     truncation=True,
#     max_length=512,
#     return_tensors="pt",
# )
# torch.onnx.export(
#     model,
#     dummy_model_input["input_ids"],
#     "bert1.onnx",
#     opset_version=12,
#     input_names=["input_ids"],
#     output_names=["output"],
#     dynamic_axes={
#         "input_ids": {
#             0: "batch_size",
#             1: "sequence_len"
#         },
#         "output": {
#             0: "batch_size"
#         }
#     }
# )

In [5]:
# onnx_model = onnx.load("bert1.onnx")
# onnx.checker.check_model(onnx_model)

In [6]:
# ort_session = onnxruntime.InferenceSession("bert1.onnx")

In [8]:
# def to_numpy(tensor):
#        return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()


In [1]:
# # compute ONNX Runtime output prediction
# ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
# ort_outs = ort_session.run(None, ort_inputs)